# ESPN scraping

Taking a look at the match commentary data.

In [3]:
!*go mod edit -replace "github.com/umbralcalc/trywizard=/home/robert/Code/trywizard"

In [6]:
import (
  "github.com/umbralcalc/trywizard/pkg/espn"
)

%% 

// TODO: load/derive list of gameIDs for this season
gameIDs := []string{"600014"}

for _, gid := range gameIDs {
  events, err := espn.FetchEvents(gid)
  if err != nil {
    log.Printf("Error fetching %s: %v\n", gid, err)
    continue
  }
  fmt.Printf("=== Events for game %s ===\n", gid)
  for _, e := range events {
    fmt.Println(e)
  }
  fmt.Println()
}

=== Events for game 600014 ===
{0 {1 try} 5 0 {1} {307 6'} {0 6'} {25903  s:300~t:25903 Leicester Tigers LEICS Leicester Tigers   [] []} [{{296838 Jack van Poortvliet}}]}
{0 {2 conversion} 7 0 {1} {359 6'} {0 6'} {25903  s:300~t:25903 Leicester Tigers LEICS Leicester Tigers   [] []} [{{165132 Handre Pollard}}]}
{0 {3 penalty goal} 7 3 {1} {549 10'} {0 10'} {25898  s:300~t:25898 Bath Rugby BATH Bath Rugby   [] []} [{{158721 Finn Russell}}]}
{0 {1 try} 7 8 {1} {1600 27'} {0 27'} {25898  s:300~t:25898 Bath Rugby BATH Bath Rugby   [] []} [{{227779 Thomas du Toit}}]}
{0 {2 conversion} 7 10 {1} {1657 28'} {0 28'} {25898  s:300~t:25898 Bath Rugby BATH Bath Rugby   [] []} [{{158721 Finn Russell}}]}
{0 {5 yellow card} 7 10 {1} {1700 29'} {0 29'} {25903  s:300~t:25903 Leicester Tigers LEICS Leicester Tigers   [] []} [{{226027 Julian Montoya}}]}
{0 {7 player substituted} 7 10 {1} {1794 30'} {0 30'} {25903  s:300~t:25903 Leicester Tigers LEICS Leicester Tigers   [] []} [{{290928 Tommy Reffell}}]}
